# Vertical stream function conservation

author, Wenrui Jiang 06/06/2022

This notebook is also an idealized test for 2D flow like [the horizontal case](./hor_stream). However, it has a slightly different flavor

1. This notebook looks at the vertical direction rather than the horizontal one. 
2. We are going to simulate the particles backward in time. 
3. The "wall" behavior of the particles are explored. 

In [ ]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import seaduck as sd
import warnings
warnings.filterwarnings("ignore")

## Loading data

As always, we need to load the data. For simplicity and to demonstrate the capacity of particles sliding on the wall, I am going to use the stream function of f-plane gyre. You can think of this example as a zonal overturning cell in a meridional channel. 

In [ ]:
N = 100
M = 50
x = np.linspace(-1,1,N+1)
y = np.linspace(-0.1,0.1,2)
zl = np.linspace(0,-1000,M)
zp1 = np.append(zl,-1001)
xg,yg = np.meshgrid(x,y)
xv = 0.5*(xg[:,1:]+xg[:,:-1])
yv = 0.5*(yg[:,1:]+yg[:,:-1])
xu = 0.5*(xg[1:]+xg[:-1])
yu = 0.5*(yg[1:]+yg[:-1])

xc = 0.5*(xv[1:]+xv[:-1])
yc = 0.5*(yv[1:]+yv[:-1])

tempx,tempz = np.meshgrid(x,np.linspace(-1,1,M))
strmf = ((np.cos(np.pi*tempx/2)*np.cos(np.pi*tempz/2))).reshape(len(zl),1,-1)
z = 0.5*(zp1[1:]+zp1[:-1])
zl = zp1[:-1]
drf = np.abs(np.diff(zp1))

In [ ]:
u = np.zeros((M,1,N+1),float)
u[:-1] = np.diff(strmf,axis = 0)
w = np.diff(strmf,axis = -1)
v = np.zeros((M,2,N),float)

In [ ]:
ds = xr.Dataset(
    coords = dict(
        XC = (["Y","X"],xc),
        YC = (["Y","X"],yc),
        XG = (["Yp1","Xp1"],xg),
        YG = (["Yp1","Xp1"],yg),
        Zl = (["Zl"],zl),
        Z  = (["Z"],z),
        drF = (["Z"],drf),
        rA = (["Y","X"],np.ones_like(xc,float))
    ),
    data_vars = dict(
        UVELMASS = (["Z","Y","Xp1"],u),
        VVELMASS = (["Z","Yp1","X"],v),
        WVELMASS = (["Zl","Y","X"],w),
    )
)

## Prepare the test

First, we convert the `xarray.Dataset` to `seaduck.OceData`

In [ ]:
tub = sd.OceData(ds)

Now we define the initial condition of particles such that some particles are right **on the wall/at the corner**.

In [ ]:
n = 30
m = 30
x = np.linspace(-1,1,n+1)
z = np.linspace(-0.1,-1000,m+1)
x,z = np.meshgrid(x,z)
x = x.ravel()
z = z.ravel()

Let's store which ones are on the wall and which ones are at the corner for plotting purposes. 

In [ ]:
on_x_wall = np.abs(x)==1
on_z_wall = np.logical_or(z == z.min(),z == z.max())
coloring = (0.2+np.logical_or(on_x_wall,on_z_wall))/1.2
corner = np.logical_and(on_x_wall,on_z_wall)

## Run the test and plot
Finally, we can run the simulation, and plot the trajectories. 

In [ ]:
pt = sd.Particle(x = x,y = np.zeros_like(x),z = z,t = np.zeros_like(x),data = tub, transport = True)

In [ ]:
steps =40
stops,ps = pt.to_list_of_time(normal_stops = np.linspace(0,-2*steps*N,steps),update_stops = [])

In [ ]:
lons = []
deps = []
for ppp in ps:
    lons.append(ppp.lon)
    deps.append(ppp.dep)
lons = np.array(lons)
deps = np.array(deps)

In [ ]:
from matplotlib import cm, colors
norm = colors.Normalize(vmin=0.0, vmax=1.0, clip=True)
mapper = cm.ScalarMappable(norm=norm, cmap=cm.binary)

In [ ]:
for i in range(len(x)):
    if corner[i]:
        plt.plot(lons.T[i],deps.T[i],'x-',c = 'r')
    plt.plot(lons.T[i],deps.T[i],c = mapper.to_rgba(coloring[i]))
plt.ylabel('Depth')
plt.xlabel('X')
plt.title('Particle trajectories in a zonal overturning cell')
plt.show()

**Fig.1** Particle trajectories in zonal overturning cell. Grey and black lines represent the trajectories of interior and wall attached particles, respectively. The red crosses show the trajectories of corner points. 

## Remarks
Hopefully, this pattern has already convinced you that the paticles are conserving streamfunction in the vertical. 

The non-penetrating condition means that particles on the wall will not detach in any finite time. Likewise, particles at the corners will never leave. 

At the center, the point with maximum stream function has no where else to go. 